In [1]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.image import resize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import json
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Завантаження
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Нормалізація та розширення до 3 каналів
x_train = np.stack([x_train] * 3, axis=-1) / 255.0
x_test = np.stack([x_test] * 3, axis=-1) / 255.0

# Масштабування зображень до 224x224 (для VGG16)
x_train = np.array([resize(img, (32, 32)) for img in x_train])
x_test = np.array([resize(img, (32, 32)) for img in x_test])

# one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
image_input = tf.keras.layers.Input(shape=(32,32, 3)) 
baseModel_VGG_16 = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_tensor=image_input) 

for layer in baseModel_VGG_16.layers:
    layer.trainable = False

baseModel_VGG_16.summary()  

for layer in baseModel_VGG_16.layers:
    layer.trainable = True

In [ ]:
model_final = Sequential([
    baseModel_VGG_16,
    Flatten(),
    Dense(units=1000, activation='relu'),
    Dense(units=800, activation='relu'),
    Dense(units=400, activation='relu'),
    Dense(units=200, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=10, activation='softmax')
])

model_final.summary()

In [ ]:
base_learning_rate = 0.0001  
optimizer = Adam(learning_rate=base_learning_rate, clipvalue=1.0)
model_final.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# training the Model with call back along with test and training data on a batch size of 128
history = model_final.fit(np.asarray(x_train),
                          np.asarray(y_train),
                          validation_split=0.1,
                          epochs=5,
                          batch_size=32) 

In [ ]:
# Сохранение истории в файл
with open('vgg16_history.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:


with open('vgg16_history.json', 'r') as f:
    history_data = json.load(f)

# Построение графика точности
plt.plot(history_data['accuracy'], label='Точность на обучающем наборе')
plt.plot(history_data['val_accuracy'], label='Точность на валидационном наборе')
plt.xlabel('Эпохи')
plt.ylabel('Точность')
plt.legend()
plt.title('График точности')
plt.show()

# Построение графика потерь
plt.plot(history_data['loss'], label='Потери на обучающем наборе')
plt.plot(history_data['val_loss'], label='Потери на валидационном наборе')
plt.xlabel('Эпохи')
plt.ylabel('Потери')
plt.legend()
plt.title('График потерь')
plt.show()

In [ ]:
model_final.save('fashion_mnist_fine_tuned_vgg16.h5')
model_final.save_weights('fashion_mnist_fine_tuned_vgg16_weights.h5')

Висновки:
VGG16 - складніша модель, дає дуже хороші результати, але навчання займає багато часу, і вона може бути перевантаженою для простих задач.
Згорткова модель -  швидше навчається і дає схожі результати, але з меншою складністю. 
Вона може бути хорошим варіантом для задач, де важлива швидкість і ефективність, але при цьому не дуже важлива точність, яка може просідати